In [2]:
# COPY

# read in dataset

import boto3
import pandas as pd
from sagemaker import get_execution_role

df = pd.read_feather("../../data/df_2016_tweets_cleaned_with_place.feather")

In [ ]:
len(df.index)

In [29]:
res_arr = []

for row in df1.iterrows():
    row[1]["place_1"] = "something"
    res_arr.append(row[1])

pd.concat(res_arr, axis=1).transpose()

,id,DateTime,tweet_content,cashtags,coordinates,hashtags,lang,like_count,media,mentioned_users,...,user_profile_banner_url,user_label,resident/visitor,tweet,place_full_name,place_name,place_type,place_country,place_country_code,place_1
0,743561801745338368,2016-06-16 21:52:02,😁 https://t.co/eq2i7lxQkD,None,"Coordinates(longitude=-85.3044586, latitude=8....",None,und,1,[Photo(previewUrl='https://pbs.twimg.com/media...,None,...,https://pbs.twimg.com/profile_banners/59929935...,None,visitor,,"Puntarenas, Costa Rica",Puntarenas,admin,Costa Rica,CR,something
1,743064119754919936,2016-06-15 12:54:26,Morning ☕️🌲🍂🌾🍃,None,"Coordinates(longitude=-85.3044586, latitude=8....",None,en,0,None,None,...,https://pbs.twimg.com/profile_banners/59929935...,None,visitor,morning,"Puntarenas, Costa Rica",Puntarenas,admin,Costa Rica,CR,something
2,742934385624809472,2016-06-15 04:18:55,👬👭❤️🤗 https://t.co/nMQNrePMXZ,None,None,None,und,0,[Photo(previewUrl='https://pbs.twimg.com/media...,None,...,https://pbs.twimg.com/profile_banners/59929935...,None,visitor,,None,None,None,None,None,something
3,726430125286264832,2016-04-30 15:16:52,Se van los 25 :/ https://t.co/RddGa6gT4T,None,None,None,es,3,[Photo(previewUrl='https://pbs.twimg.com/media...,None,...,https://pbs.twimg.com/profile_banners/59929935...,None,visitor,se van los 25 :/,None,None,None,None,None,something
4,684363985974464512,2016-01-05 13:21:03,Godo Morning 👓,None,"Coordinates(longitude=-84.851172, latitude=9.9...",None,en,1,None,None,...,https://pbs.twimg.com/profile_banners/59929935...,None,visitor,godo morning,"Puntarenas, Costa Rica",Puntarenas,city,Costa Rica,CR,something
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,668796156995072000,2015-11-23 14:20:04,"Buen día, bendiciones de Dios y éxitos!!! (@ C...",None,"Coordinates(longitude=-83.92678536, latitude=9...",None,es,0,[Photo(previewUrl='https://pbs.twimg.com/media...,None,...,None,None,visitor,"buen da, bendiciones de dios y xitos!!! ( casa...","Cartago, Costa Rica",Cartago,city,Costa Rica,CR,something
96,668162263853961216,2015-11-21 20:21:12,Santa Misa! Dar infinitas gracias a Dios por t...,None,"Coordinates(longitude=-83.92220593, latitude=9...",None,es,0,[Photo(previewUrl='https://pbs.twimg.com/media...,None,...,None,None,visitor,santa misa! dar infinitas gracias a dios por t...,"Cartago, Costa Rica",Cartago,city,Costa Rica,CR,something
97,665625924474376192,2015-11-14 20:22:41,Santa Misa! Dar gracias a Dios 🙏🏼🙏🏼🙏🏼👍🏼 (@ Cap...,None,"Coordinates(longitude=-83.92220593, latitude=9...",None,es,0,[Photo(previewUrl='https://pbs.twimg.com/media...,None,...,None,None,visitor,santa misa! dar gracias a dios ( capilla del h...,"Cartago, Costa Rica",Cartago,city,Costa Rica,CR,something
98,664032978448867328,2015-11-10 10:52:53,"Buen dia, Dios les bendiga (@ Casa-Cartago.) h...",None,"Coordinates(longitude=-83.92678536, latitude=9...",None,es,0,[Photo(previewUrl='https://pbs.twimg.com/media...,None,...,None,None,visitor,"buen dia, dios les bendiga ( casa-cartago.)","Cartago, Costa Rica",Cartago,city,Costa Rica,CR,something


In [42]:
df1 = df[0:100]


from multiprocessing import Pool # multithreading
import tqdm
import re


def extract_coordinates_helper(row):
    if row[1]["coordinates"] != "None":
        try:
            split_by = "Coordinates\(longitude=|, latitude=|\)"
            temp = re.split(split_by, row[1]["coordinates"])

            row[1]['coordinates_longitude'] = temp[1]
            row[1]['coordinates_latitude'] = temp[2]

        except:
            row[1]['coordinates_longitude'] = None
            row[1]['coordinates_latitude'] = None

    return row[1]


# extracting place components
def extract_coordinates(df):
    
    df_coord = df
    
    df_coord['coordinates_longitude'] = "None"
    df_coord['coordinates_latitude'] = "None"
    
    
    pool = Pool(processes=len(df_coord.index))

    result_arr = []
    
    for result in tqdm.tqdm(pool.imap_unordered(extract_coordinates_helper, df_coord.iterrows()),
                            total=len(df_coord.index)):
        result_arr.append(result)
                
    df_coord = pd.concat(result_arr, axis=1).transpose().sort_index()
                
    return df_coord


df2 = extract_coordinates(df1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

100%|██████████| 100/100 [00:00<00:00, 4797.44it/s]


In [43]:
df1.coordinates.value_counts()

None                                                        30
Coordinates(longitude=-84.5919495, latitude=9.0655537)      16
Coordinates(longitude=-85.4598007, latitude=10.1362932)      9
Coordinates(longitude=-83.930542, latitude=9.8601866)        9
Coordinates(longitude=-83.92678536, latitude=9.86399942)     5
Coordinates(longitude=-83.92220593, latitude=9.86148165)     5
Coordinates(longitude=-83.9428711, latitude=9.0631237)       5
Coordinates(longitude=-84.851172, latitude=9.9700296)        4
Coordinates(longitude=-85.3044586, latitude=8.0379929)       2
Coordinates(longitude=-84.0791779, latitude=9.4674778)       2
Coordinates(longitude=-83.79956175, latitude=9.91726612)     1
Coordinates(longitude=-87.0792369, latitude=5.2863477)       1
Coordinates(longitude=-84.6279891, latitude=9.6440191)       1
Coordinates(longitude=-84.0093726, latitude=9.8990788)       1
Coordinates(longitude=-84.0093131, latitude=9.8988487)       1
Coordinates(longitude=-84.0094316, latitude=9.8988592) 

In [44]:
df2.coordinates_longitude.value_counts(dropna=False)

None            30
-84.5919495     16
-85.4598007      9
-83.930542       9
-83.92678536     5
-83.92220593     5
-83.9428711      5
-84.851172       4
-85.3044586      2
-84.0791779      2
-83.79956175     1
-87.0792369      1
-84.6279891      1
-84.0093726      1
-84.0093131      1
-84.0094316      1
-84.202605       1
-84.2025895      1
-84.2029705      1
-84.0167096      1
-84.0095134      1
-84.0094562      1
-83.7451336      1
Name: coordinates_longitude, dtype: int64

In [3]:
# seeing language distribution of tweets
df.lang.value_counts(dropna = False)
# most of them are in spanish (understandably as this was the original search parameter)
# and it would be a lot more work translating from multiple languages, so we just filter out
# tweets that aren't in spanish, but also keep english ones

es     5598666
en      874472
und     441829
pt      125326
zxx      83202
ca       68466
qme      67087
in       56222
fr       53781
it       41321
tl       38741
art      37610
ht       30650
qht      14607
eu       14588
qst      14304
et       12779
qam       9672
de        9402
cy        9132
sv        7652
nl        7564
ro        6769
tr        6485
hu        6392
th        6360
da        6153
lt        6112
pl        4835
cs        3940
fi        3639
no        2885
lv        2782
ja        2261
is        2106
sl        1893
ru        1589
hi        1400
vi        1229
ar        1111
zh        1019
ko         353
uk          82
el          32
bg          17
iw          12
fa           6
sr           4
kn           2
hy           1
mr           1
Name: lang, dtype: int64

In [4]:
# filtering out all tweets that aren't in spanish
df_es = df[df.lang == "es"]

# creating separate df with english tweets
df_en = df[df.lang == "en"]

In [5]:
df1 = df_es

df1.loc[df1.coordinates != "None","coordinates"] = "Yes"  
df1.coordinates.value_counts(dropna=False)

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


None    4317143
Yes     1281523
Name: coordinates, dtype: int64

https://huggingface.co/docs/transformers/tasks/translation

In [6]:
# deploying model from HuggingFace

from sagemaker.huggingface import HuggingFaceModel
import sagemaker

role = sagemaker.get_execution_role()
# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'Helsinki-NLP/opus-mt-es-en',
	'HF_TASK':'translation'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.17.0',
	pytorch_version='1.10.2',
	py_version='py38',
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge' # ec2 instance type
)

predictor.predict({
	'inputs': "Me llamo Bob."
})

-----!

[{'translation_text': 'My name is Bob.'}]

In [13]:
! pip show pandas

Name: pandas
Version: 1.3.5
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /opt/conda/lib/python3.7/site-packages
Requires: numpy, python-dateutil, pytz
Required-by: autovizwidget, hdijupyterutils, sagemaker, sagemaker-data-insights, seaborn, smclarify, sparkmagic, statsmodels


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Register `pandas.progress_apply` and `pandas.Series.map_apply` with `tqdm`
# (can use `tqdm.gui.tqdm`, `tqdm.notebook.tqdm`, optional kwargs, etc.)

df_es["tweet_en"] = df_es.apply(lambda x: predictor.predict({"inputs": x['tweet_content']})[0]['translation_text'], axis=1)
df_es["tweet_en"]

In [50]:
q = df1.tweet_content.tolist()[0:10]

In [55]:
df.place.value_counts(dropna=False)

None                                                                                                                                                        511647
Place(fullName='Rosario, Argentina', name='Rosario', type='city', country='Argentina', countryCode='AR')                                                     43528
Place(fullName='Ciudad Autónoma de Buenos Aires, Argentina', name='Ciudad Autónoma de Buenos Aires', type='city', country='Argentina', countryCode='AR')     39017
Place(fullName='Buenos Aires, Argentina', name='Buenos Aires', type='admin', country='Argentina', countryCode='AR')                                          21395
Place(fullName='Córdoba, Argentina', name='Córdoba', type='admin', country='Argentina', countryCode='AR')                                                    12712
                                                                                                                                                             ...  
Place(fullName='Costa 

In [57]:
predictor.predict({"inputs": df.tweet_content.tolist()[0:100]})

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from primary with message "Your invocation timed out while waiting for a response from container primary. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://eu-west-2.console.aws.amazon.com/cloudwatch/home?region=eu-west-2#logEventViewer:group=/aws/sagemaker/Endpoints/huggingface-pytorch-inference-2022-12-27-16-01-36-180 in account 937484630899 for more information.

In [ ]:
df1['tweet_content_en'] = english_tweets
df1[['tweet_content', 'tweet_content_en']]

In [ ]:
import feather

df.to_feather("../../data/df_2016.feather")
boto3.Session().resource('s3').Bucket("mt5599").Object("tweets/df_2016.feather").upload_file("../../data/df_2016.feather")